__Import__

*   Importieren Sie _fipy_ und benennen Sie das Paket `fp`
*   Fügen Sie zusätzlich
```python
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
```
    hinzu
*   Importieren Sie noch weitere Pakete (z.B. Numpy, ...) nachträglich, falls Sie diese benötigen.

<!--
import fipy as fp
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
-->

__Netz- und Variablenerstellung__

*    Erstellen Sie ein zweidimensionales Gitter `mesh` in einem quadratischen Gebiet mit der Kantenlänge `1` und diskretisieren Sie jede Richtung mit `20` Elementen
*    Definieren Sie Zellvariablen für den Druck $p$, das Druckkorrekturfeld $p'$, die Geschwindigkeit in $x$-Richtung $u_x$, die Geschwindigkeit in $y$-Richtung $u_y$ und das jeweilige Volumen der Zelle $V_C$ als `P`, `PCorr`, `Ux`, `Uy` und `V_C`

<!--
L = 1.0
N = 20
dL = L / N

mesh = fp.Grid2D(nx=N, ny=N, dx=dL, dy=dL)

P = fp.CellVariable(mesh=mesh)
PCorr = fp.CellVariable(mesh=mesh)
Ux = fp.CellVariable(mesh=mesh)
Uy = fp.CellVariable(mesh=mesh)

V_C = fp.CellVariable(mesh=mesh, value=mesh.cellVolumes)
-->

*    Erstellen Sie zusätzlich eine Zellvariable mit `rank=1` für die linearisierten Geschwindigkeiten durch
```python
UTilde = fp.CellVariable(mesh=mesh, rank=1)
```

<!--
UTilde = fp.CellVariable(mesh=mesh, rank=1)
-->

__Impulsgleichung__

*   Die Diskretisierung der Impulsgleichung wird durch eine Aufwind-Diskretisierung durchgeführt
*   Beim iterativen Lösen der Matrix wird zusätzlich eine Relaxation durchgeführt, d.h. es wird ein gewichtetes Mittel aus Alt- und Neuwert bestimmt entsprechend $x^{(n)}=\omega x^{(n)} + (1-\omega) x^{(n-1)}$
*   Wählen Sie einen geeigneten Löser und übergeben Sie diesen als Attribut `solver` den beiden [`sweep`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.terms.html#fipy.terms.term.Term.sweep)-Methoden
*   Es stehen z.B. die beiden Löser [`LinearBicgstabSolver`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.solvers.scipy.html#module-fipy.solvers.scipy.linearBicgstabSolver) und [`LinearPCGSolver`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.solvers.scipy.html#module-fipy.solvers.scipy.linearPCGSolver) in `fp.solvers.scipy` zur Verfügung
*   Machen Sie sich klar, welchen Termen die beiden Zellvariablen als Rückgabewerte der Funktionen `giveDiagA11` und `giveDiagA22` auf der [Folie 9 (Seite 13/30)](https://ilias3.uni-stuttgart.de/goto_Uni_Stuttgart_file_2962285_download.html) entsprechen.
```python
class momentumEquation:
        def __init__(self, rho, mu, UTilde):
          divU = UTilde[0,:].grad.dot([1.0,0.0])+UTilde[1,:].grad.dot([0.0,1.0])
          self.A11_ = \
            rho * fp.UpwindConvectionTerm(coeff=UTilde) - fp.DiffusionTerm(coeff=mu) \
            == \
            - P.grad.dot([1.0,0.0]) \
            + mu * divU.grad.dot([1.0,0.0]) - rho * fp.ImplicitSourceTerm(coeff=divU)
          self.A22_ = \
            rho * fp.UpwindConvectionTerm(coeff=UTilde) - fp.DiffusionTerm(coeff=mu) \
            == \
            - P.grad.dot([0.0,1.0]) \
            + mu * divU.grad.dot([0.0,1.0]) - rho * fp.ImplicitSourceTerm(coeff=divU)
          self.A11_.cacheMatrix()
          self.A22_.cacheMatrix()

        def solve(self, Ux, Uy, relax):  
          self.xres_ = self.A11_.sweep(
            var=Ux, 
            underRelaxation=relax, 
            solver=...
          )
          self.yres_ = self.A22_.sweep(
            var=Uy, 
            underRelaxation=relax, 
            solver=...
          )

          return [Ux,Uy]

        def giveDiagA11(self):
          return fp.CellVariable(mesh=mesh, value=fp.numerix.asarray(self.A11_.matrix.takeDiagonal()))

        def giveDiagA22(self):
          return fp.CellVariable(mesh=mesh, value=fp.numerix.asarray(self.A22_.matrix.takeDiagonal()))
```

<!--
class momentumEquation:
    def __init__(self, rho, mu, UTilde):
      divU = UTilde[0,:].grad.dot([1.0,0.0])+UTilde[1,:].grad.dot([0.0,1.0])
      self.A11_ = \
        rho * fp.UpwindConvectionTerm(coeff=UTilde) - fp.DiffusionTerm(coeff=mu) \
        == \
        - P.grad.dot([1.0,0.0]) \
        + mu * divU.grad.dot([1.0,0.0]) - rho * fp.ImplicitSourceTerm(coeff=divU)
      self.A22_ = \
        rho * fp.UpwindConvectionTerm(coeff=UTilde) - fp.DiffusionTerm(coeff=mu) \
        == \
        - P.grad.dot([0.0,1.0]) \
        + mu * divU.grad.dot([0.0,1.0]) - rho * fp.ImplicitSourceTerm(coeff=divU)
      self.A11_.cacheMatrix()
      self.A22_.cacheMatrix()
        
    def solve(self, Ux, Uy, relax):
      self.A11_.sweep(
        var=Ux, 
        underRelaxation=relax, 
        solver=fp.solvers.scipy.LinearBicgstabSolver()
      )
      self.A22_.sweep(
        var=Uy, 
        underRelaxation=relax, 
        solver=fp.solvers.scipy.LinearBicgstabSolver()
      )
        
      return [Ux,Uy]
    
    def giveDiagA11(self):
      return fp.CellVariable(mesh=mesh, value=fp.numerix.asarray(self.A11_.matrix.takeDiagonal()))
    
    def giveDiagA22(self):
      return fp.CellVariable(mesh=mesh, value=fp.numerix.asarray(self.A22_.matrix.takeDiagonal()))
-->

__Rhie-Chow-Interpolation__

*    Erstellen Sie eine Klasse `rhieChowInterpolation`
*    Der Konstruktor soll als Eingangsargumente die beiden Terme `Dx` und `Dy` entsprechend [Folie 19 (Seite 25/30)](https://ilias3.uni-stuttgart.de/goto_Uni_Stuttgart_file_2962285_download.html) besitzen
*    Die Interpolation des Drucks soll in der Methode `interpolate` erfolgen
```python
    def interpolate(self, Ux, Uy, P, U_f):  
        gradP = P.grad
        gradP_f = fp.variables.faceGradVariable._FaceGradVariable(P)

       U_f[0] = ...
       U_f[1] = ...
    
       return U_f
```
*   Fügen Sie `interpolate` als Methode der Klasse hinzu
*   `U_f` ist eine [Zellflächenvariable](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.variables.html#module-fipy.variables.faceVariable)
*   Die Variable `gradP` entspricht dem Druckgradient im Zellzentrum
*   Die Variable `gradP_f` entspricht dem Druckgradienten auf den Zellflächen
*   Die Variablen im Zellzentrum können Sie durch die Klassenfunktion [arithmeticFaceValue](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.variables.html#fipy.variables.cellVariable.CellVariable.arithmeticFaceValue) auf die Zellflächen interpolieren
*   Vervollständigen Sie die Rhie-Chow-Interpolation (Formel auf [Folie 19 (Seite 25/30)](https://ilias3.uni-stuttgart.de/goto_Uni_Stuttgart_file_2962285_download.html)) und berechnen Sie die Geschwindigkeit `U_f` auf den Zellflächen

<!--
class rhieChowInterpolation:
  def __init__(self, Dx, Dy):
    self.Dx_ = Dx
    self.Dy_ = Dy
        
  def interpolate(self, Ux, Uy, P, U_f):  
    gradP = P.grad
    gradP_f = fp.variables.faceGradVariable._FaceGradVariable(P)

    U_f[0] = Ux.arithmeticFaceValue - self.Dx_.arithmeticFaceValue * (gradP_f[0]-gradP[0].arithmeticFaceValue)
    U_f[1] = Uy.arithmeticFaceValue - self.Dy_.arithmeticFaceValue * (gradP_f[1]-gradP[1].arithmeticFaceValue)
    
    return U_f
-->

__Druckkorrekturgleichung__

*    Erstellen Sie eine Klasse `pressureCorrectionEquation`
*    Die Druckkorrekturgleichung von [Folie 13](https://ilias3.uni-stuttgart.de/goto_Uni_Stuttgart_file_2509497_download.html) lautet anders geschrieben $\nabla (\nabla D p') = \nabla \cdot u$
*    Verwenden Sie zur Berechnung der rechten Seite das Attribut [`divergence`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.variables.html#fipy.variables.faceVariable.FaceVariable.divergence) von `U_f`
*    Lösen Sie die Gleichung für die Variable `PCorr` mit der Methode [`sweep`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.terms.html#fipy.terms.term.Term.sweep)
*    Wählen Sie einen geeigneten Löser

<!--
class pressureCorrectionEquation:
    import numpy as np
    def __init__(self):
        pass
    
    def solve(self, PCorr, D, U_f):
      pressureCorrectionEq = fp.DiffusionTerm(coeff=D) == U_f.divergence
      pressureCorrectionEq.solve(
        var=PCorr, 
        solver=fp.solvers.scipy.LinearPCGSolver()
      )
      return PCorr
-->

__Physikalische Eigenschaften und Randbedingungen__

*    Definieren Sie $\mu$ und $\rho$ als `mu` und `rho` und belegen Sie sie mit `0.1` und `1.0`
*    Definieren Sie zusätzlich einen Skalar `U` als Konstante für die Randbedingungen und belegen Sie die Variable mit `1.0`
*    Berechnen Sie die Reynolds-Zahl

<!--
mu = 0.1
rho = 1.0
U = 1.0
print("Re = ", U * L / mu * rho)
-->

*   Initialisieren Sie die Felder `Ux`, `Uy`, `P` und `PCorr` mit `0`
*   Setzen Sie für `Ux` und `Uy` Dirichlet Randbedingungen an allen Flächen `mesh.exteriorFaces`
*   Für die Geschwindigkeit `Uy` gilt überall Haftbedingung
*   An der oberen Fläche `mesh.facesTop` liegt für `Ux` die Geschwindigkeit `U` an

<!--
Ux.setValue(0.0)
Uy.setValue(0.0)
P.setValue(0.0)
PCorr.setValue(0.0)
Ux.constrain(0.0, mesh.facesRight | mesh.facesLeft | mesh.facesBottom)
Ux.constrain(U, mesh.facesTop)
Uy.constrain(0.0, mesh.exteriorFaces)
-->

*   Für den Druck `PCorr` wird am Punkt links unten eine Dirichlet Randbedingung vorgegeben:
```python
  X, Y = mesh.faceCenters
  PCorr.constrain(0., (Y <= np.min(Y)+dL/3) & (X <= np.min(X)+dL/3) )
  P.constrain(0., (Y <= np.min(Y)+dL/3) & (X <= np.min(X)+dL/3) )
```

* Zusätzlich wird auf alle Randflächen eine Neumann-Randbedingung gesetzt:
```python
  PCorr.faceGrad.constrain(((0,),(0,)), where=mesh.exteriorFaces)
```
<!--
X, Y = mesh.faceCenters
PCorr.constrain(0., (Y <= np.min(Y)+dL/3) & (X <= np.min(X)+dL/3) )
P.constrain(0., (Y <= np.min(Y)+dL/3) & (X <= np.min(X)+dL/3) )
PCorr.faceGrad.constrain(((0,),(0,)), where=mesh.exteriorFaces)
-->

__SIMPLE-Algorithmus__

Das Grundgerüst des SIMPLE-Algorithmus ist:
```python
  # initialize
  [...]
  for iteration in range(iterations+1):
    # linearize
    [...]
    # solve momentum equation
    [...]
    # rhie-chow-interpolation
    [...]
    # correct boundary conditions
    [...]
    # pressure correction
    [...]
    # update pressure and velocity
    [..]
    # visualize
    [...]
```

*    Kopieren Sie das Grundgerüst

_#initialize_
*    Fügen Sie die Befehle
```python
  plt.close(1)
  plt.close(2)
  plt.close(3)
  iterations = 25
  relaxP = 0.3
  relaxU = 1.-relaxP
```
ein

_#linearize_
*    Belegen Sie die Variable `UTilde` mit dem alten Geschwindigkeitsfeld `[Ux.value,Uy.value]` durch [setValue](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.variables.html#fipy.variables.variable.Variable.setValue)

_#solve momentum equations_
*    Erstellen Sie ein Objekt `mEq` der Klasse `momentumEquation`
*    Verwenden Sie als Relaxationsparameter den Wert `0.5`
*    Lösen Sie die Impulsgleichung und belegen Sie die beiden Felder `Ux` und `Uy` neu
*    Berechnen Sie das Residuum für die Impulsgleichung in $x$- und $y$-Richtung bzw. $u_{x,res}$ und $u_{y,res}$ entsprechend $u_{x,res} = \sum |u_x - \tilde{u}_x|$ und $u_{x,res} = \sum |u_x - \tilde{u}_x|$
*    Speichern Sie das Ergebnis in den Variablen `uxRes` und `uyRes`

_#rhie-chow-interpolation_
*    Erstellen Sie eine Zellflächenvariable:
```python
  U_f = fp.FaceVariable(mesh=mesh, rank=1)
```
*    Berechnen Sie die beiden Terme `Dx` und `Dy`
*    Interpolieren Sie die Geschwindigkeiten `Ux` und `Uy` und belegen Sie das Feld `U_f`

_#correct boundary conditions_
*     Auf dem Geschwindigkeitsfeld `U_f` müssen die Randbedingungen ebenfalls definiert werden
```python
    U_f[0].constrain(0, mesh.exteriorFaces.value)
    U_f[1].constrain(0, mesh.exteriorFaces.value)
    U_f[0].constrain(U, mesh.facesTop.value)
```

_#pressure correction equation_
*   Erstellen Sie ein Objekt `pEq` der Klasse `pressureCorrectionEquation`
*   Lösen Sie die Gleichung und verwenden Sie als Diffusionskonstante $\Gamma_p$ das Feld `np.sqrt(Dx**2+Dy**2)`
*   Speichern Sie den neuen Druck in `PCorr`
*   Belegen Sie die Variable `cRes` mit einem geeigneten Residuum zur Konvergenzbewertung des Druckes

_#update pressure and velocity_
*   Aktualisieren Sie den Druck durch
```python
  P.setValue( P + relaxP * PCorr )
```
    und die Geschwindigkeit durch
```python
  Ux.setValue(Ux - relaxU * PCorr.grad[0]  * Dx)
  Uy.setValue(Uy - relaxU * PCorr.grad[1]  * Dy)
```

_#visualize_
*    Die Visualisierung der Lösung erfolgt in jeder `25`-ten Iteration
*    Das Residuum wird in jeder Iteration in ein Schaubild gezeichnet
```python
    if not iteration%25:        
        plt.figure(1)
        plt.clf()
        plt.plot(
          np.array(mesh.cellCenters[0]), 
          np.array(mesh.cellCenters[1]),
          'k.'
        )
        cntr2 = plt.tricontourf(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            np.sqrt(np.array(Ux.value)**2+np.array(Uy.value)**2), 
            levels=np.arange(0.0, 0.8, 0.02),
            cmap="jet"
        )        
        plt.colorbar(cntr2)
        plt.quiver(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            np.array(Ux.value), np.array(Uy.value),
            edgecolor='k', facecolor='None', linewidth=.5
        )        
        plt.gcf().canvas.draw()

        plt.figure(2)
        plt.clf()
        cntr3 = plt.tricontourf(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            P.value, levels=np.arange(-5.0, 10.0, 1.0), cmap="jet"
        )
        plt.colorbar(cntr3)
        plt.gcf().canvas.draw()

    plt.figure(3)
    plt.semilogy(iteration, uxRes, 'r.')
    plt.semilogy(iteration, uyRes, 'g.')
    plt.semilogy(iteration, cRes, 'ks')
    plt.gcf().canvas.draw()
```

<!--
# initialize
plt.close(1)
plt.close(2)
plt.close(3)
iterations = 25
relaxP = 0.3
relaxU = 1.-relaxP
for iteration in range(iterations+1):
    # linearize
    UTilde.setValue([Ux.value,Uy.value])

    # solve momentum equation
    mEq = momentumEquation(rho, mu, UTilde)
    [Ux,Uy] = mEq.solve(Ux, Uy, 0.5)
   
    uxRes = np.sum(np.abs(np.array(Ux[:] - UTilde[0][:])))
    uyRes = np.sum(np.abs(np.array(Uy[:] - UTilde[1][:])))

    # rhie-chow-interpolation
    U_f = fp.FaceVariable(mesh=mesh, rank=1)
    Dx = V_C / mEq.giveDiagA11()
    Dy = V_C / mEq.giveDiagA22()
    U_f = rhieChowInterpolation(Dx, Dy).interpolate(Ux, Uy, P, U_f)#, UTilde[0], UTilde[1], relaxP)
    
    # correct boundary conditions
    U_f[0].constrain(0, mesh.exteriorFaces.value)
    U_f[1].constrain(0, mesh.exteriorFaces.value)
    U_f[0].constrain(U, mesh.facesTop.value)
    U_f[1].constrain(0, mesh.facesTop.value)
    
    # pressure correction
    pEq = pressureCorrectionEquation()

    PCorr = pEq.solve(PCorr, np.sqrt(Dx**2.+Dy**2.) / V_C, U_f)
    cRes = np.sum(np.abs(PCorr.value)) / np.sum(np.abs(P.value))
    
    # update pressure and velocity
    P.setValue( P + relaxP * PCorr )
    Ux.setValue(Ux - relaxU * PCorr.grad[0] * Dx)
    Uy.setValue(Uy - relaxU * PCorr.grad[1] * Dy)

    # visualize
    if not iteration%25:        
        plt.figure(1)
        plt.clf()
        cntr2 = plt.tricontourf(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            np.sqrt(np.array(Ux.value)**2+np.array(Uy.value)**2), 
            levels=np.arange(0,1.001,0.1),
            cmap="jet"
        )
        plt.colorbar(cntr2)
        plt.quiver(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            np.array(Ux.value), np.array(Uy.value),
            edgecolor='k', facecolor='None', linewidth=.5
        )
        plt.gcf().canvas.draw()
        
        plt.figure(2)
        plt.clf()
        cntr3 = plt.tricontourf(
            np.array(mesh.cellCenters[0]), 
            np.array(mesh.cellCenters[1]), 
            P.value, levels=np.arange(-4.0,4.001,0.5), 
            cmap="jet"
        )
        plt.colorbar(cntr3)
        plt.gcf().canvas.draw()
        
    plt.figure(3)
    plt.semilogy(iteration, uxRes, 'r.')
    plt.semilogy(iteration, uyRes, 'g.')
    plt.semilogy(iteration, cRes, 'ks')
    plt.gcf().canvas.draw()
-->

__Spielen Sie__

*    Bitte spielen Sie mit dem Löser
*    Erhöhen Sie die Anzahl der Iterationen, ändern Sie die Relaxationsparameter, Ändern Sie die Reynoldszahl, Vergrößern oder Verkleinern Sie das Netz, ...
*    __Tipp__: der Cavity-Flow hat nach etwa 250 Iterationen Residuen $<10^-1$

**Wichtiger Hinweis**

Bitte machen Sie das Netz nicht zu groß. Bitte verwenden Sie nicht mehr als $100x100$ Elemente.
